# Exercise 1 | Inspecting a Bitcoin address

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import json
from pprint import pprint
    
import graphsense
from graphsense.api import addresses_api, bulk_api, entities_api, general_api

## Notebook Setup

This notebook assumes that you already followed the instructions outlined in `README.md` and installed the [GraphSense Python API](https://github.com/graphsense/graphsense-python) library locally.

First, we must setup our notebook and establish a connection to some hosted GraphSense instance.

We connect to a GraphSense instance hosted by [Iknaio](https://www.ikna.io/) and enter our API key in the provided `config.json` configuration file. An API key will be provided during the tutorial. If you would like to get an API key later, drop an email to contact@iknaio.com

## Load host and API key from config

In [19]:
f = open('config.json')
config = json.load(f)
f.close()

## Configure GraphSense client

Now we configure the GraphSense Python library

In [20]:
configuration = graphsense.Configuration(
    host = config['graphsense']['host'],
    api_key = {'api_key': config['graphsense']['api_key']})

We can test if our libary is working by retrieving summary statistics on supported ledgers.

In [21]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = general_api.GeneralApi(api_client)
    api_response = api_instance.get_statistics()
    pprint(api_response)

{'currencies': [{'name': 'btc',
                 'no_address_relations': 8838400431,
                 'no_addresses': 1198080403,
                 'no_blocks': 810968,
                 'no_entities': 539074172,
                 'no_labels': 18757,
                 'no_tagged_addresses': 286973150,
                 'no_txs': 903675306,
                 'timestamp': 1696626939},
                {'name': 'bch',
                 'no_address_relations': 2483597490,
                 'no_addresses': 341485142,
                 'no_blocks': 814258,
                 'no_entities': 153580428,
                 'no_labels': 154,
                 'no_tagged_addresses': 15922605,
                 'no_txs': 375181820,
                 'timestamp': 1696832873},
                {'name': 'ltc',
                 'no_address_relations': 1790574705,
                 'no_addresses': 208110517,
                 'no_blocks': 2559048,
                 'no_entities': 90626455,
                 'no_labels': 242,

# Address API

We start by retrieving some summary statistics on the Internet Archive's donation address. The response basically corresponds to the data shown in the property box of the GraphSense dashboard.

In [22]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = addresses_api.AddressesApi(api_client)

    currency = "btc"
    address = '1Archive1n2C579dMsAu3iC6tWzuQJz8dN'
    include_tags = True

    try:
        resp_addr = api_instance.get_address(currency, address)
        pprint(resp_addr)
    except graphsense.ApiException as e:
        print("Exception when calling AddressesApi->get_address: %s\n" % e)

{'actors': [{'id': 'internet_archive', 'label': 'Internet Archive'}],
 'address': '1Archive1n2C579dMsAu3iC6tWzuQJz8dN',
 'balance': {'fiat_values': [{'code': 'eur', 'value': 33395.54},
                             {'code': 'usd', 'value': 35275.71}],
             'value': 126225428},
 'currency': 'btc',
 'entity': 2647118,
 'first_tx': {'height': 256314,
              'timestamp': 1378415426,
              'tx_hash': '04d92601677d62a985310b61a301e74870fa942c8be0648e16b1db23b996a8cd'},
 'in_degree': 7313,
 'last_tx': {'height': 810390,
             'timestamp': 1696298264,
             'tx_hash': 'bd9e5225e5e500996b4406faebb6d41d7f13308cfab3be81c27de0aa96a14f78'},
 'no_incoming_txs': 5483,
 'no_outgoing_txs': 287,
 'out_degree': 302,
 'status': 'clean',
 'total_received': {'fiat_values': [{'code': 'eur', 'value': 6554328.0},
                                    {'code': 'usd', 'value': 6987914.5}],
                    'value': 62081260018},
 'total_spent': {'fiat_values': [{'code': 'eur'

In [23]:
print(f"Address {resp_addr.address} received {resp_addr.total_received.fiat_values[0].value} EUR " + 
      f"from {resp_addr.in_degree} addresses.")

Address 1Archive1n2C579dMsAu3iC6tWzuQJz8dN received 6554328.0 EUR from 7313 addresses.


# Entities API

Next, we inspect the corresponding entity, which clusters other addresses that are likely conrolled by the owner of that address. The entity ID is contained in the address response.

In [24]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = entities_api.EntitiesApi(api_client)

    currency = "btc" 
    entity = resp_addr.entity 
    include_tags = True 

    try:
        resp_entity = api_instance.get_entity(currency, entity)
        pprint(resp_entity)
    except graphsense.ApiException as e:
        print("Exception when calling EntitiesApi->get_entity: %s\n" % e)

{'balance': {'fiat_values': [{'code': 'eur', 'value': 33395.54},
                             {'code': 'usd', 'value': 35275.71}],
             'value': 126225428},
 'best_address_tag': {'actor': 'internet_archive',
                      'address': '1Archive1n2C579dMsAu3iC6tWzuQJz8dN',
                      'category': 'organization',
                      'confidence': 'service_data',
                      'confidence_level': 50,
                      'currency': 'BTC',
                      'entity': 2647118.0,
                      'is_cluster_definer': True,
                      'label': 'internet archive',
                      'lastmod': 1636675200,
                      'source': 'https://archive.org/donate/cryptocurrency',
                      'tagpack_creator': 'GraphSense Core Team',
                      'tagpack_is_public': True,
                      'tagpack_title': 'GraphSense Demo TagPack',
                      'tagpack_uri': '1f030eb44c7ca37b:packs/demo.yaml'},
 'cu

In [25]:
print(f"Entity {resp_entity.entity} likely controls {resp_entity.no_addresses} addresses, " +
      f"which received {resp_entity.total_received.fiat_values[0].value} EUR " + 
      f"from {resp_entity.in_degree} entities.")

Entity 2647118 likely controls 124 addresses, which received 8766926.0 EUR from 5743 entities.


# Entity Neighbors

Finally, we retrieve all neighbor entities receiving funds from our focus entity. Since it is more convenient to work with dataframes, we use the GraphSense Bulk API interface to retrieve a dataframe of receiving neighbor nodes.

In [26]:
import pandas as pd

data = {'entity': [resp_addr.entity]}

entityDF = pd.DataFrame.from_dict(data)

In [27]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = bulk_api.BulkApi(api_client)

    CURRENCY = "btc"
    operation = "list_entity_neighbors"
    body = {'entity': entityDF['entity'].to_list(),
            'direction': 'out',
            'include_labels': True}

    try:
        respDF = pd.read_csv(
                    api_instance.bulk_csv(CURRENCY, operation, body=body,
                                          num_pages=2,
                                          _preload_content=False))
    except graphsense.ApiException as e:
        print("Exception when calling BulkApi->bulk_csv: %s\n" % e)

In [28]:
respDF

,_error,_info,_request_entity,entity_actors,entity_balance_eur,entity_balance_usd,entity_balance_value,entity_best_address_tag_abuse,entity_best_address_tag_actor,entity_best_address_tag_address,...,entity_total_spent_value,entity_total_tokens_received,entity_total_tokens_spent,labels,labels_count,no_txs,token_values,value_eur,value_usd,value_value
0,NaN,NaN,2647118,NaN,2.343859e+08,2.475819e+08,885910592215,NaN,NaN,1CsBZRX2CpCg2Bk1tDyCCMYoS8WgeSijCA,...,5227463228862170,NaN,NaN,"bitcoin-24.com,coinjoin mess,epay.info,supercl...",5,2,NaN,750.20,986.69,953789282
1,NaN,NaN,2647118,NaN,5.014494e+04,5.296810e+04,189533269,NaN,bitpaycom,12cGCaeGXgjDsmsksw22ScGatxWqjbb4Vh,...,133520683745741,NaN,NaN,bitpay.com,1,1,NaN,537.80,673.60,200000000
2,NaN,NaN,2647118,NaN,3.339554e+04,3.527571e+04,126225428,NaN,internet_archive,1Archive1n2C579dMsAu3iC6tWzuQJz8dN,...,170501423571,NaN,NaN,internet archive,1,107,NaN,147862.66,182066.95,37095153475
3,NaN,NaN,2647118,NaN,0.000000e+00,0.000000e+00,0,NaN,NaN,NaN,...,48610000000,NaN,NaN,NaN,0,1,NaN,2411.06,3074.92,48610000000
4,NaN,NaN,2647118,NaN,4.275434e+07,4.516141e+07,161598938582,NaN,coinbase,335umQ4egqMjDo6wSXYwQZYPntZzMveNTJ,...,1312172790500063,NaN,NaN,"coinbase,coinbase hot wallet,coinbase pro,coin...",4,77,NaN,11922.25,13517.07,262142450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,NaN,NaN,2647118,NaN,0.000000e+00,0.000000e+00,0,NaN,NaN,NaN,...,600000000,NaN,NaN,NaN,0,1,NaN,171694.66,203131.97,600000000
174,NaN,NaN,2647118,NaN,0.000000e+00,0.000000e+00,0,NaN,NaN,NaN,...,618131591,NaN,NaN,NaN,0,1,NaN,277.73,328.77,1000000
175,NaN,NaN,2647118,NaN,0.000000e+00,0.000000e+00,0,NaN,NaN,NaN,...,18899997228,NaN,NaN,NaN,0,1,NaN,4077640.50,4097621.25,18899997228
176,NaN,NaN,2647118,NaN,3.700000e-01,3.900000e-01,1381,NaN,NaN,NaN,...,0,NaN,NaN,NaN,0,1,NaN,0.29,0.29,1382


We filter the result and see the relations between known entity IDs. We also see two known receiving entities.

In [29]:
resultDF = respDF[['_request_entity', 'entity_entity', 'labels', 'no_txs']] \
    .rename(columns={'_request_entity': 'src', 'entity_entity': 'dst', 'labels': 'dst_label'}) \
    .dropna()
resultDF

,src,dst,dst_label,no_txs
0,2647118,109578,"bitcoin-24.com,coinjoin mess,epay.info,supercl...",2
1,2647118,1084056,bitpay.com,1
2,2647118,2647118,internet archive,107
4,2647118,4635851,"coinbase,coinbase hot wallet,coinbase pro,coin...",77
107,2647118,319021678,"bitpanda,kraken,kraken deposit wallet,kraken.com",5
141,2647118,391306062,bitpay deposit wallet,1
